## Preprocess

**Import train and test datasets**

In [256]:
import pandas as pd
from lightgbm import LGBMClassifier
import numpy as np

train_data = pd.read_csv("input_data/train.csv")
test_data = pd.read_csv("input_data/test.csv")


In [220]:
train_data.info()
print(train_data.count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB
id          230130
date        230130
country     230130
store       230130
product     230130
num_sold    221259
dtype: int64


**Process data for training and testing**

In [261]:
data["num_sold"].count(np.nan)

TypeError: Series.count() takes 1 positional argument but 2 were given

In [277]:
def get_processed_data(data:pd.DataFrame):
    from datetime import datetime

    data = data.copy()
    
    weekday = [
        datetime(
                int(date.split("-")[0]),
                int(date.split("-")[1]),
                int(date.split("-")[2])
            ).strftime("%A")
            for date in data["date"].copy()
        ]

    data = data.drop('date', axis=1)

    data["weekday"] = weekday

    if "num_sold" not in data.keys():
        data["num_sold"] = [0 for i in data["id"]]

    data = data.drop('id', axis=1)

    data["num_sold"] = data["num_sold"].fillna(0)
    train_df = data.dropna().reset_index(drop=True)
    X, y = train_df.drop('num_sold',axis=1), train_df.num_sold

    for feat in X.select_dtypes(exclude='O').columns:
        X[feat].fillna(X[feat].median())
        X[feat+'_bin'] = pd.qcut(X[feat],q=20,duplicates='drop',labels=False)
    cat_feat = X.select_dtypes(include='O').columns.tolist()
    X[cat_feat] = X[cat_feat].fillna('NA')
    X = pd.get_dummies(X,columns=cat_feat,drop_first=True,dtype=int)

    return (X, y)

train_X, train_y = get_processed_data(train_data)

test_X, _ = get_processed_data(test_data)

list(train_X.columns)

['country_Finland',
 'country_Italy',
 'country_Kenya',
 'country_Norway',
 'country_Singapore',
 'store_Premium Sticker Mart',
 'store_Stickers for Less',
 'product_Kaggle',
 'product_Kaggle Tiers',
 'product_Kerneler',
 'product_Kerneler Dark Mode',
 'weekday_Monday',
 'weekday_Saturday',
 'weekday_Sunday',
 'weekday_Thursday',
 'weekday_Tuesday',
 'weekday_Wednesday']

In [263]:
print(train_y)

0            0.0
1          973.0
2          906.0
3          423.0
4          491.0
           ...  
230125     466.0
230126    2907.0
230127    2299.0
230128    1242.0
230129    1622.0
Name: num_sold, Length: 230130, dtype: float64


**Create model**

In [264]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(max_depth=10, n_estimators=1000, learning_rate=0.01, reg_alpha=1, reg_lambda=1, scale_pos_weight=3.5, verbose=-1, objective="regression")

**Train model**

In [265]:
import numpy as np

def mean_cross_validation_scores(score):
    mean_scores = {metric: np.mean(values) for metric, values in score.items()}
    return pd.DataFrame(mean_scores, index=['Score'])

In [266]:
from sklearn.model_selection import cross_validate

#score = cross_validate(model, train_X, train_y, cv=5, scoring=['roc_auc'])
score = cross_validate(model, train_X, train_y, cv=5, scoring=['neg_mean_squared_error', 'r2'])


In [267]:
mean_cross_validation_scores(score)

,fit_time,score_time,test_neg_mean_squared_error,test_r2
Score,2.03632,0.337169,-19559.11122,0.958341


In [268]:
model.fit(train_X,train_y)

LGBMRegressor(learning_rate=0.01, max_depth=10, n_estimators=1000,
              objective='regression', reg_alpha=1, reg_lambda=1,
              scale_pos_weight=3.5, verbose=-1)

In [269]:
# train_X['pred_prob'] = model.predict_proba(train_X)[:,1]
# test_X['pred_prob'] = model.predict_proba(test_X)[:,1]

In [270]:
# pred = model.predict_proba(test_X)[:,1]

train_X['raw_score'] = model.predict(train_X, raw_score=True)

# Convert raw scores to probabilities
train_X['pred_prob'] = 1 / (1 + np.exp(-train_X['raw_score']))

# Repeat for the test set
test_X['raw_score'] = model.predict(test_X, raw_score=True)
test_X['pred_prob'] = 1 / (1 + np.exp(-test_X['raw_score']))

In [271]:
test_X['raw_score']
test_X['pred_prob']

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
98545    1.0
98546    1.0
98547    1.0
98548    1.0
98549    1.0
Name: pred_prob, Length: 98550, dtype: float64

In [272]:
submission = pd.DataFrame({"id": test_data['id'], "num_sold": test_X['raw_score']})
submission.to_csv("submission.csv", index=False)

In [283]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings("ignore")

# Constants
RANDOM_SEED = 42

# # Load data
# train_data = pd.read_csv("input_data/train.csv")
# test_data = pd.read_csv("input_data/test.csv")

# # Drop rows where 'num_sold' is missing in training data
# train_data = train_data.dropna(subset=['num_sold']).reset_index(drop=True)

# # Separate features and target
# X = train_data.drop(columns=['num_sold', 'id'])
# y = train_data['num_sold']
# X_test = test_data.drop(columns=['id'])

# Minimal preprocessing: Handle categorical variables with one-hot encoding
def preprocess_data(df):
    df = pd.get_dummies(df, drop_first=True)
    return df

X = preprocess_data(X)
X_test = preprocess_data(X_test)

# Align columns between train and test sets
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# Hyperparameter optimization

def objective(trial):
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 0.9),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 0.9),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
        "objective": "regression",
        "metric": "mape",
        "boosting_type": "gbdt",
        "random_state": RANDOM_SEED,
        "n_jobs": -1,
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)
    rmse_scores = []

    for train_idx, valid_idx in kf.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = lgb.LGBMRegressor(**param, n_estimators=100)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric="mape",
            callbacks=[
                    lgb.early_stopping(stopping_rounds=10),  # Corrected early stopping\n",
                    lgb.log_evaluation(100)
                    ],

        )

        preds = model.predict(X_valid)
        rmse_scores.append(mean_absolute_percentage_error(y_valid, preds, squared=False))

    return np.mean(rmse_scores)

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # Reduced number of trials for faster execution

# Train final model with best parameters
best_params = study.best_params
best_params.update({
    "objective": "regression",
    "metric": "mape",
    "boosting_type": "gbdt",
    "random_state": RANDOM_SEED,
    "n_estimators": 200,  # Increase for final training
})

final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(X, y)

# Predict on test data
y_pred = final_model.predict(X_test)

# Save predictions
submission = pd.DataFrame({"id": test_data['id'], "num_sold": y_pred})
submission.to_csv("submission.csv", index=False)

print("Submission file created.")

[I 2025-01-16 20:16:06,023] A new study created in memory with name: no-name-6219b445-a65c-4e69-964f-82f883a5a0a4


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[53]	valid_0's mape: 0.764771


[W 2025-01-16 20:16:17,052] Trial 0 failed with parameters: {'learning_rate': 0.08916653004464412, 'num_leaves': 42, 'max_depth': 5, 'feature_fraction': 0.6247445523150107, 'bagging_fraction': 0.7186398020604705, 'bagging_freq': 3, 'lambda_l1': 4.5742493366539385, 'lambda_l2': 0.5000765154904369} because of the following error: TypeError("got an unexpected keyword argument 'squared'").
Traceback (most recent call last):
  File "C:\Users\SpaldingBananaClub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\SpaldingBananaClub\AppData\Local\Temp\ipykernel_9684\1159378177.py", line 77, in objective
    rmse_scores.append(mean_absolute_percentage_error(y_valid, preds, squared=False))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\

TypeError: got an unexpected keyword argument 'squared'

In [288]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings("ignore")

# Constants
RANDOM_SEED = 42

# Load data
train_data = pd.read_csv("input_data/train.csv")
test_data = pd.read_csv("input_data/test.csv")

# Drop rows where 'num_sold' is missing in training data
train_data = train_data.dropna(subset=['num_sold']).reset_index(drop=True)

# Separate features and target
X = train_data.drop(columns=['num_sold', 'id'])
y = train_data['num_sold']
X_test = test_data.drop(columns=['id'])

# Minimal preprocessing: Handle categorical variables with one-hot encoding
def preprocess_data(df):
    df = pd.get_dummies(df, drop_first=True)
    return df

X = preprocess_data(X)
X_test = preprocess_data(X_test)

# Align columns between train and test sets
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# Hyperparameter optimization

def objective(trial):
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 0.9),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 0.9),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
        "objective": "regression",
        "metric": "mape",
        "boosting_type": "gbdt",
        "random_state": RANDOM_SEED,
        "n_jobs": -1,
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)
    mape_scores = []

    for train_idx, valid_idx in kf.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = lgb.LGBMRegressor(**param, n_estimators=100)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric="mape",
            callbacks=[lgb.early_stopping(stopping_rounds=20)] # increase for heavier training
        )

        preds = model.predict(X_valid)
        mape_scores.append(mean_absolute_percentage_error(y_valid, preds))

    return np.mean(mape_scores)

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)  # Reduced number of trials for faster execution

# Train final model with best parameters
best_params = study.best_params
best_params.update({
    "objective": "regression",
    "metric": "mape",
    "boosting_type": "gbdt",
    "random_state": RANDOM_SEED,
    "n_estimators": 200,  # Increase for final training
})

final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(X, y)

# Predict on test data
y_pred = final_model.predict(X_test)

# Save predictions
submission = pd.DataFrame({"id": test_data['id'], "num_sold": y_pred})
submission.to_csv("submission.csv", index=False)

print("Submission file created.")


[I 2025-01-16 22:33:47,833] A new study created in memory with name: no-name-cab2fb47-c228-4e14-a2f8-7e221609deee


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[62]	valid_0's mape: 0.333844
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[69]	valid_0's mape: 0.327701
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[69]	valid_0's mape: 0.325691


[I 2025-01-16 22:34:22,017] Trial 0 finished with value: 0.3290785703861205 and parameters: {'learning_rate': 0.07528814593789598, 'num_leaves': 30, 'max_depth': 9, 'feature_fraction': 0.7620274449195754, 'bagging_fraction': 0.6177824262639452, 'bagging_freq': 4, 'lambda_l1': 4.015369617015364, 'lambda_l2': 0.35993399051323316}. Best is trial 0 with value: 0.3290785703861205.


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[39]	valid_0's mape: 0.435572
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[45]	valid_0's mape: 0.388853
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[42]	valid_0's mape: 0.415795


[I 2025-01-16 22:34:55,156] Trial 1 finished with value: 0.41340638911262767 and parameters: {'learning_rate': 0.11119614485273865, 'num_leaves': 29, 'max_depth': 5, 'feature_fraction': 0.8112713050966351, 'bagging_fraction': 0.7038358797213802, 'bagging_freq': 1, 'lambda_l1': 2.1806338198918156, 'lambda_l2': 4.902980364519581}. Best is trial 0 with value: 0.3290785703861205.


...

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[30]	valid_0's mape: 0.187126
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[35]	valid_0's mape: 0.196783
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[30]	valid_0's mape: 0.183512


[I 2025-01-17 07:52:36,664] Trial 998 finished with value: 0.1891402200863923 and parameters: {'learning_rate': 0.19248990704520635, 'num_leaves': 32, 'max_depth': 10, 'feature_fraction': 0.8628922304004097, 'bagging_fraction': 0.8045026523613994, 'bagging_freq': 5, 'lambda_l1': 0.15715015325231568, 'lambda_l2': 2.187580053503238}. Best is trial 287 with value: 0.1784714710342492.


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[34]	valid_0's mape: 0.197079
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[39]	valid_0's mape: 0.203638
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[37]	valid_0's mape: 0.202116


[I 2025-01-17 07:53:10,212] Trial 999 finished with value: 0.20094456054635848 and parameters: {'learning_rate': 0.19300239049621476, 'num_leaves': 33, 'max_depth': 9, 'feature_fraction': 0.8824362245019783, 'bagging_fraction': 0.8774744095607544, 'bagging_freq': 5, 'lambda_l1': 0.31139405974920187, 'lambda_l2': 1.9789232309054394}. Best is trial 287 with value: 0.1784714710342492.


Submission file created.
